In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
plt.style.use('ggplot')
%matplotlib inline

In [2]:
with open('Manual WorkItems alltypes Security.csv', encoding='ascii', errors='ignore') as infile:
    dataset1 = pd.read_csv(infile,names = ['Title', 'Security', 'SourceLink'])
with open('Manual WorkItems alltypes nonSecurity.csv', encoding='ascii', errors='ignore') as infile:
    dataset2 = pd.read_csv(infile,names = ['Title', 'Security', 'SourceLink'])

In [3]:
dataset1 = dataset1.sample(frac = 0.45, random_state= 123).reset_index(drop = True)
dataset1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63262 entries, 0 to 63261
Data columns (total 3 columns):
Title         63261 non-null object
Security      63262 non-null object
SourceLink    63262 non-null object
dtypes: object(3)
memory usage: 1.4+ MB


In [4]:
dataset1['IsSecured'] = 1
dataset2['IsSecured'] = 0

In [5]:
dataset = pd.concat([dataset1, dataset2]).reset_index(drop = True)
dataset.head()

,Title,Security,SourceLink,IsSecured
0,[WDGISScan] Perf.Service.PASAPI 'pasdb.westus....,MS.Security,https://microsoft.visualstudio.com/DefaultColl...,1
1,[Studio]Form shouldn't load another datasource...,MS.Security,https://msazure.visualstudio.com/DefaultCollec...,1
2,Glusterfs should only allow certain ip range t...,MS.Security,https://msasg.visualstudio.com/DefaultCollecti...,1
3,[dev15->devmain] Fuzzing: Mac Word: doc: Proba...,MS.Security,http://bugcheck/bugs/OfficeMain/75542,1
4,(Port to wacserver 16) MSRC 32783. Word Crash ...,MS.Security,https://office.visualstudio.com/DefaultCollect...,1


In [6]:
dataset = dataset.sample(frac = 1).reset_index(drop = True)
dataset.head()

,Title,Security,SourceLink,IsSecured
0,no ForceReconsent for implicit offers,MS.Security,https://identitydivision.visualstudio.com/Defa...,1
1,[Home Page][BugBash] Missing Screen shot - no ...,MS.Security,https://msazure.visualstudio.com/DefaultCollec...,1
2,iwm_setup_wmsparameters contains unnecessary c...,MS.Privacy,https://msazure.visualstudio.com/DefaultCollec...,0
3,MMTV: Image Cuts: Promotions: Peaky Blinders - S4,MS.GlobalReadiness,https://microsoft.visualstudio.com/DefaultColl...,0
4,[XP][WS/Partner/SQL][VARIANT CASE of 36465] [M...,MS.Security,https://microsoft.visualstudio.com/DefaultColl...,1


In [7]:
dataset.dropna(axis=0, how='any', inplace = True)

In [8]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122058 entries, 0 to 122059
Data columns (total 4 columns):
Title         122058 non-null object
Security      122058 non-null object
SourceLink    122058 non-null object
IsSecured     122058 non-null int64
dtypes: int64(1), object(3)
memory usage: 4.7+ MB


In [9]:
from nltk.corpus import stopwords
my_stopwords = stopwords.words('english')
type(my_stopwords)

list

In [10]:
from patsy import dmatrices
import statsmodels.discrete.discrete_model as sm1
from statsmodels.formula.api import logit, ols, poisson, probit
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
import os, sys
def splitall(path):
    allparts = []
    while 1:
        parts = os.path.split(path)
        if parts[0] == path:  # sentinel for absolute paths
            allparts.insert(0, parts[0])
            break
        elif parts[1] == path: # sentinel for relative paths
            allparts.insert(0, parts[1])
            break
        else:
            path = parts[0]
            allparts.insert(0, parts[1])
    return " ".join(allparts)

In [12]:
#I will now apply the splitall function on the dataset.
dataset['Parsed Title'] = dataset['Title'].apply(splitall)
dataset.head(100)

,Title,Security,SourceLink,IsSecured,Parsed Title
0,no ForceReconsent for implicit offers,MS.Security,https://identitydivision.visualstudio.com/Defa...,1,no ForceReconsent for implicit offers
1,[Home Page][BugBash] Missing Screen shot - no ...,MS.Security,https://msazure.visualstudio.com/DefaultCollec...,1,[Home Page][BugBash] Missing Screen shot - no ...
2,iwm_setup_wmsparameters contains unnecessary c...,MS.Privacy,https://msazure.visualstudio.com/DefaultCollec...,0,iwm_setup_wmsparameters contains unnecessary c...
3,MMTV: Image Cuts: Promotions: Peaky Blinders - S4,MS.GlobalReadiness,https://microsoft.visualstudio.com/DefaultColl...,0,MMTV: Image Cuts: Promotions: Peaky Blinders - S4
4,[XP][WS/Partner/SQL][VARIANT CASE of 36465] [M...,MS.Security,https://microsoft.visualstudio.com/DefaultColl...,1,[XP][WS Partner SQL][VARIANT CASE of 36465] [M...
5,[MSRC 30781] [VBScript] [5.6] [Srv2003] vbscri...,MS.Security,https://microsoft.visualstudio.com/DefaultColl...,1,[MSRC 30781] [VBScript] [5.6] [Srv2003] vbscri...
6,[Education][GDI] - periodic element - pt-br - ...,MS.GlobalReadiness,https://msasg.visualstudio.com/DefaultCollecti...,0,[Education][GDI] - periodic element - pt-br - ...
7,HMDS - use APSecrets for azure storage keys,MS.Security,https://msasg.visualstudio.com/DefaultCollecti...,1,HMDS - use APSecrets for azure storage keys
8,Semmle: shell/twinui/VirtualDesktops/lib/Virtu...,MS.Security,https://microsoft.visualstudio.com/DefaultColl...,1,Semmle: shell twinui VirtualDesktops lib Virtu...
9,Caret is not moving ahead after enabling caret...,MS.Accessibility,https://microsoft.visualstudio.com/DefaultColl...,0,Caret is not moving ahead after enabling caret...


In [13]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
def clean_text(text):
    text = "".join([char if char.isalpha() else " " for char in text.lower()]).split()
    text = [ps.stem(word) for word in text]
    text = " ".join([word for word in text if (word not in my_stopwords and len(word) > 2)])
    return text.lower()

In [14]:
%time dataset['Clean Title'] = dataset['Parsed Title'].apply(clean_text)

Wall time: 50.1 s


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.feature_selection import SelectPercentile, f_classif
X, y = dataset['Clean Title'], dataset['IsSecured']
tv = TfidfVectorizer(min_df=0.001, max_df= 1.0, use_idf=True,stop_words = 'english' )
#tv = TfidfVectorizer(sublinear_tf = True, max_df = 0.5, stop_words = 'english')
#selector = SelectPercentile(f_classif, percentile = 10)
#selector.fit(tv.fit_transform(X))
X_train, X1, y_train, y1 = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X1, y1, test_size=0.5, random_state=42)
X_train = tv.fit_transform(X_train)#.toarray()
X_test = tv.transform(X_test)#.toarray()
X_val = tv.transform(X_val)#.toarray()
vocab = tv.get_feature_names()
# y_test = np.array(list(y_test))
# y_train = np.array(list(y_train))
# y_val = np.array(list(y_val))
print(len(vocab))


1099


In [16]:
X2 = pd.DataFrame(X_train.toarray(), columns = vocab)
X2["Intercept"] = 1.0

X3 = pd.DataFrame(X_val.toarray(), columns = vocab)
X3["Intercept"] = 1.0

X4 = pd.DataFrame(X_test.toarray(), columns = vocab)
X4["Intercept"] = 1.0

vocab = vocab +['Intercept']

In [17]:
X2.head()

,aad,abil,abl,acc,accept,access,account,aci,action,activ,...,write,wrong,xbox,xml,xss,year,zero,zone,zoom,Intercept
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.283785,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [18]:
X2["IsSecured"] = list(y_train)

In [19]:
from pandas.core import datetools
import statsmodels.api as sm

C:\Users\inpganavar\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  """Entry point for launching an IPython kernel.


In [20]:
%time logit = sm.Logit(X2['IsSecured'], X2[vocab])

Wall time: 25.2 s


In [34]:
# fit the model
%time result = logit.fit(method='bfgs',maxiter=100)

         Current function value: 0.187219
         Iterations: 100
         Function evaluations: 102
         Gradient evaluations: 102
Wall time: 38 s


C:\Users\inpganavar\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [35]:
# cool enough to deserve it's own gist
result.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                                      Results: Logit
==========================================================================================
Model:                       Logit                    Pseudo R-squared:         0.730     
Dependent Variable:          IsSecured                AIC:                      29619.5722
Date:                        2018-04-10 12:19         BIC:                      39731.9274
No. Observations:            73234                    Log-Likelihood:           -13711.   
Df Model:                    1098                     LL-Null:                  -50716.   
Df Residuals:                72135                    Scale:                    1.0000    
Converged:                   0.0000                                                       
------------------------------------------------------------------------------------------
                        Coef.      Std.Err.      z     P>|z|      [0.025         0.975]   
------------------------------------------------------------------------------------------
aad                      0.4213        0.5500   0.7660 0.4437        -0.6567        1.4993
abil                    -0.2673        0.6090  -0.4389 0.6607        -1.4609        0.9263
abl                     -0.0707        0.4756  -0.1488 0.8817        -1.0029        0.8614
acc                     -4.7251        0.9879  -4.7830 0.0000        -6.6613       -2.7888
accept                  -1.5215        0.5746  -2.6479 0.0081        -2.6476       -0.3953
access                  -7.9227        0.2558 -30.9736 0.0000        -8.4240       -7.4214
account                  1.1318        0.3125   3.6215 0.0003         0.5193        1.7443
aci                      1.9508        0.9598   2.0326 0.0421         0.0697        3.8319
action                  -0.9678        0.4289  -2.2563 0.0241        -1.8084       -0.1271
activ                   -0.5375        0.3599  -1.4936 0.1353        -1.2429        0.1678
actual                   1.4616        0.9782   1.4942 0.1351        -0.4556        3.3789
adapt                   -2.4137        0.7635  -3.1615 0.0016        -3.9101       -0.9174
add                     -0.1079        0.2268  -0.4755 0.6344        -0.5525        0.3367
addit                   -1.6161        0.5845  -2.7650 0.0057        -2.7617       -0.4705
address                  0.3172        0.4657   0.6812 0.4958        -0.5955        1.2300
adjust                   1.5829        0.6661   2.3765 0.0175         0.2775        2.8884
admin                    0.8070        0.3945   2.0457 0.0408         0.0338        1.5801
administr                1.4184        0.7697   1.8427 0.0654        -0.0903        2.9271
adminport                5.0992        1.2061   4.2280 0.0000         2.7354        7.4631
advanc                   1.7456        0.6725   2.5955 0.0094         0.4274        3.0638
afd                      4.7835        1.3780   3.4714 0.0005         2.0827        7.4842
aft                      4.0855        1.2787   3.1950 0.0014         1.5793        6.5917
agent                   -0.8065        0.4464  -1.8066 0.0708        -1.6814        0.0685
aip                     -0.6802        1.0240  -0.6643 0.5065        -2.6872        1.3267
album                   -2.4131        0.9780  -2.4674 0.0136        -4.3299       -0.4963
alert                    0.9186        0.3852   2.3850 0.0171         0.1637        1.6735
align                    0.3850        0.5023   0.7665 0.4434        -0.5995        1.3695
allow                    1.8773        0.4779   3.9280 0.0001         0.9406        2.8141
alreadi                  0.8582        0.7655   1.1211 0.2622        -0.6421        2.3586
alt                     -6.1755        1.0101  -6.1139 0.0000        -8.1552       -4.1958
altern                  -0.6314        0.7667  -0.8236 0.4102        -2.1340        0.8712
alway                    0.7773        0.6356   1.2230 0.2213        -0.4685

In [ ]:
#Converting the type to float64

In [50]:
!pip install pandas_ml

  Running setup.py bdist_wheel for pandas-ml: started
  Running setup.py bdist_wheel for pandas-ml: finished with status 'done'
  Stored in directory: C:\Users\inpganavar\AppData\Local\pip\Cache\wheels\13\57\a6\6c0dc8cffa09bb45bfb0b4f88fc13871342bac6a6394784f83
Successfully built pandas-ml


  Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x000000000494FF60>: Failed to establish a new connection: [Errno 11004] getaddrinfo failed',)': /simple/pandas-ml/
  Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x000000000494FEF0>: Failed to establish a new connection: [Errno 11004] getaddrinfo failed',)': /simple/pandas-ml/
  Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x000000000494F438>: Failed to establish a new connection: [Errno 11004] getaddrinfo failed',)': /simple/pandas-ml/
You are using pip version 9.0.1, however version 9.0.3 is available

In [42]:
from pandas_ml import ConfusionMatrix
from sklearn.metrics import confusion_matrix

predict=result.predict(X3)

Y_val_predict=[]
for i in predict:
    if(i>=0.5):
        Y_val_predict.append(1)
    else:
        Y_val_predict.append(0)

conf=confusion_matrix(y_val,Y_val_predict)
#plt.imshow(conf, cmap='binary', interpolation='None')
print(conf)
#plt.show()

cm = ConfusionMatrix(y_val, Y_val_predict)
cm.print_stats()




[[10640   989]
 [  882 11901]]
population: 4839
P: 2536
N: 2303
PositiveTest: 2534
NegativeTest: 2305
TP: 1350
TN: 1119
FP: 1184
FN: 1186
TPR: 0.5323343848580442
TNR: 0.48588797221016067
PPV: 0.5327545382794001
NPV: 0.4854663774403471
FPR: 0.5141120277898393
FDR: 0.46724546172059983
FNR: 0.4676656151419558
ACC: 0.5102293862368258
F1_score: 0.5325443786982249
MCC: 0.018221636379724526
informedness: 0.018222357068204964
markedness: 0.018220915719747266
prevalence: 0.5240752221533375
LRP: 1.035444331358172
LRN: 0.962496793272497
DOR: 1.0757899024657036
FOR: 0.514533622559653
